# Data Merge
This notebook describes steps to pull the stats from the txt files and combine them to create the final data structure.

In [142]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import os
import re
from collections import OrderedDict
from datetime import datetime


### Location Path
Define the location of files here.

In [143]:
DATA_PATH = '/Users/stereopickles/CU_Google_Drive/COLUMBIA/DSI-Schizo/Data2020/DATA040320'
HC_PATH = f"{DATA_PATH}/HC/COBRE_prep/HC_Stats"
SZ_PATH = f"{DATA_PATH}/SZ/COBRE_prep/SZ_Stats"
HC_FAT_DTI_PATH = f'{HC_PATH}/FAT'
SZ_FAT_DTI_PATH = f'{SZ_PATH}/FAT'
HC_UF_DTI_PATH = f'{HC_PATH}/UF'
SZ_UF_DTI_PATH = f'{SZ_PATH}/UF'

In [144]:
TRACTS = ['FAT', 'UF']
CONDS = ['SZ', 'HC']

### Extract stats 
The resulting table should look like 
| subnum | admin | condition | tract | datetime | side | track_count | voxel_count | mean_length | mean_FA_trk | mean_Ang | mean_FA | mean_AD | mean_MD | mean_RD | 

In [145]:
def find_num(str_):
    return int(re.findall('(?<=: )([.0-9]*)', str_)[0])

def find_mean_se(str_):
    mean = float([x if x != 'nan' else 0 for x in re.findall('(?<=: )([na.0-9]*)', str_)][0])
    se = float(re.findall('(?<=- )([.0-9]*)', str_)[0])
    return mean, se

def find_type(str_):
    return re.findall('(?<=DTI_)([A-Z]*)', str_)[0]

def find_year(str_):
    return int(re.findall('(?<=ses-)([0-9]*)', str_)[0][:4])

In [146]:
def extract_stats(list_, group = {'FAT', 'UF'}):
    ''' From a list of lines extract information '''
    result = {}
    for i in range(0, len(list_), 12):
        grp = re.findall('(?<=: )(.*)(?=\\n)', list_[i])[0]

        trk_group = re.findall('(FAT|UF)', grp)[0]
        side = grp[-1]
        if trk_group in group:
            trk_cnt = find_num(list_[i+1])
            vxl_cnt = find_num(list_[i+2])
            mean_length, se_length = find_mean_se(list_[i+4])
            mean_fatrk, se_fatrk = find_mean_se(list_[i+5])
            mean_angtrk, se_angtrk = find_mean_se(list_[i+6])
            yr_scan = find_year(list_[i+7])
            stats = {}
            for j in range(7, 11):
                stats[find_type(list_[i+j])] = find_mean_se(list_[i+j])
            if len(stats) != 4: 
                print('not enough stats')
            info = {'track_count': trk_cnt, 
                    'voxel_count': vxl_cnt, 
                    'mean_length': mean_length, 
                    'se_length': se_length, 
                    'mean_fatrk': mean_fatrk, 
                    'se_fatrk': se_fatrk, 
                    'mean_angtrk': mean_angtrk, 
                    'se_angtrk': se_angtrk, 
                    'mean_AD': stats['AD'][0], 
                    'se_AD': stats['AD'][1], 
                    'mean_FA': stats['FA'][0], 
                    'se_FA': stats['FA'][1],
                    'mean_MD': stats['MD'][0], 
                    'se_MD': stats['MD'][1],
                    'mean_RD': stats['RD'][0], 
                    'se_RD': stats['RD'][1],
                    'yr_scan': yr_scan
                   }
            if trk_group in result:
                result[trk_group][side] = info
            else:
                result[trk_group] = {side: info}
        else:
            continue
    return result

In [147]:
df = pd.DataFrame(
    columns = ['subnum', 'admin', 'condition', 'tract', 'datetime_extracted', 'side', 'note',
               'track_count', 'voxel_count', 'mean_length', 'se_length',  'mean_fatrk',
               'se_fatrk', 'mean_angtrk', 'se_angtrk', 'mean_AD', 'se_AD', 'mean_FA',
               'se_FA', 'mean_MD', 'se_MD', 'mean_RD', 'se_RD', 'yr_scan'])

In [154]:
i = 0
for tr in TRACTS:
    for cnd in CONDS: 
        path = f'{DATA_PATH}/{cnd}/COBRE_prep/{cnd}_Stats/{tr}'
        print(f'{cnd}_{tr}', end = '\n=======\n')
        try:
            sub_dirs = next(os.walk(path))[1]
        except StopIteration: 
            print("Directory doesn't exist")
            break
        for admin in sub_dirs: 
            print(f'{admin}', end = '\n-------\n')
            files = [x for x in os.listdir(os.path.join(path, admin)) if x.startswith('sub')]
            for f in files:
                if f.endswith('.txt'):
                    f2 = f
                else:
                    f2 = f + '.txt'
                subnum, note = re.findall('(?<=sub-A)([0-9]*)(.*)(?=.txt)', f2)[0]
                
                print(f'{subnum}', end = '')
                path2 = os.path.join(path, admin, f)
                dt = os.path.getmtime(path2)
                dt = datetime.fromtimestamp(dt).strftime('%Y-%m-%d %H:%M:%S')
                
                with open(path2, 'r') as fp: 
                    line = fp.readlines()
                    line = [re.sub('(?:(?<=FAT)|(?<=UF))_', '', x) for x in line]
                    info = extract_stats(line, group = {tr})
                    
                if info:
                    print('(o)', end = ' ')
                    for side in info[tr].keys():
                        vals = []
                        for k in info[tr][side].keys():
                            vals.append(info[tr][side][k])
                        full_list = [subnum, admin, cnd, tr, dt, side, note]
                        

                        full_list.extend(vals)
                        i += 1

                        df.loc[i] = full_list
                else:
                    print('(x)', end = ' ')
            print('\n')

SZ_FAT
HW
-------
00024684(o) 00038441(o) 00035836(o) 00020787(o) 00027969(o) 00037854(o) 00024568(o) 00024959(o) 00023158(o) 00038624(o) 00037649(o) 00038172(o) 00023750(o) 00027537(o) 00028189(o) 00024228(o) 00028404(o) 00027119(o) 00035859(o) 00022500(o) 00024953(o) 00024198(o) 00035485(o) 00023243(o) 00037034(o) 00020602(o) 00027391(o) 00037619(o) 00027755(o) 00037224(o) 00023246(o) 

IR
-------


SS
-------
00018598(o) 00017147(o) 00000909(o) 00001243(o) 00020414(o) 00016720(o) 00018403(o) 00014804(o) 00016197(o) 00015518(o) 00001251(o) 00014175(o) 00018317(o) 00001452(o) 00000368(o) 00009280(o) 00015648(o) 00020416(o) 00016723(o) 00014607(o) 00000456(o) 00000838(o) 00014830(o) 00014719(o) 00018129(o) 00000541(o) 00018979(o) 00019293(o) 

HC_FAT
HW
-------
00020805(o) 00021058(o) 00013363(o) 00003150(o) 00021072(o) 00018553(o) 00014225(o) 00010684(o) 00022490(o) 00011265(o) 00004087(o) 00022727(o) 00014120(o) 00022653(o) 00010150(o) 00022509(o) 00015826(o) 00014898(o) 00012995(o) 

In [116]:
df.sample(5)

,subnum,admin,condition,tract,datetime_extracted,side,note,track_count,voxel_count,mean_length,...,se_angtrk,mean_AD,se_AD,mean_FA,se_FA,mean_MD,se_MD,mean_RD,se_RD,yr_scan
78,00014804,SS,SZ,FAT,2020-12-23 15:25:47,L,,124,488,64.7097,...,4.0733,1.1267,0.2149,0.4565,0.1395,0.7416,0.1740,0.5491,0.1904,2009
102,00014607,SS,SZ,FAT,2020-12-23 15:17:42,L,,103,598,69.6019,...,5.2679,1.1143,0.1586,0.4152,0.1441,0.7612,0.1062,0.5846,0.1400,2010
79,00016197,SS,SZ,FAT,2020-12-23 15:47:26,R,,267,839,82.4569,...,4.5925,1.2293,0.3279,0.4906,0.1686,0.7756,0.2386,0.5487,0.2481,2009
101,00014607,SS,SZ,FAT,2020-12-23 15:17:42,R,,449,1297,73.5145,...,3.6949,1.1203,0.1657,0.4390,0.1414,0.7431,0.0922,0.5546,0.1229,2010
118,00019293,SS,SZ,FAT,2020-12-23 16:17:09,R,,54,395,68.1296,...,5.3987,1.1268,0.1883,0.4120,0.1339,0.7774,0.1252,0.6027,0.1443,2010


## Check for Duplicates
let's check if there are only one entry per subnum x side. If there are more than one, let's confirm that numbers are the same. If numbers are the same, only leave one.

In [124]:
entry_counts = df.groupby(['subnum', 'side']).subnum.count().unstack()

In [140]:
dupes = list(entry_counts[(entry_counts.L != 1) | (entry_counts.R != 1) ].reset_index().subnum.values)

In [141]:
df[df.subnum.isin(dupes)]

,subnum,admin,condition,tract,datetime_extracted,side,note,track_count,voxel_count,mean_length,...,se_angtrk,mean_AD,se_AD,mean_FA,se_FA,mean_MD,se_MD,mean_RD,se_RD,yr_scan
179,00007409,HW,HC,FAT,2020-11-23 21:40:52,L,(w both FAT),48,357,79.6042,...,4.3247,1.1193,0.1728,0.4472,0.1490,0.7388,0.0982,0.5485,0.1366,2011
180,00007409,HW,HC,FAT,2020-11-23 21:40:52,R,(w both FAT),75,461,81.5733,...,4.9320,1.1180,0.1389,0.4159,0.1170,0.7665,0.0867,0.5907,0.1105,2011
183,00007409,HW,HC,FAT,2020-11-23 21:34:14,L,(w both UF),70,383,76.6000,...,4.2396,1.1071,0.1785,0.4398,0.1559,0.7356,0.1015,0.5498,0.1408,2011
184,00007409,HW,HC,FAT,2020-11-23 21:34:14,R,(w both UF),137,778,83.9708,...,5.0190,1.1261,0.1549,0.4150,0.1224,0.7704,0.0905,0.5925,0.1141,2011


In [ ]:
# TODO: fix the above

## Check for Zero values
Remove zero value rows  
Add a column that indicates whether the both side exists or not

In [193]:
len(df[df.track_count == 0])

33

In [157]:
clean_df = df[df.track_count != 0]

In [178]:
tmp = clean_df.groupby(['subnum', 'tract']).subnum.count().unstack().reset_index()
missing_side = list(tmp[tmp.FAT == 1].subnum)

In [ ]:
clean_df['missing_side'] = np.where(clean_df.subnum.isin(missing_side), 1, 0)

In [194]:
cldf = clean_df[clean_df.missing_side == 0]

## Check for Outliers
Check per group per side per condition distribution
